In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv('google_api')

In [4]:
df = pd.read_csv('kinder.csv')

In [5]:
df.columns

Index(['School Name', 'Admission', 'Private/Public School',
       'Active Students in 20117'],
      dtype='object')

In [6]:
oname=['School Name', 'Admission', 'Private/Public School',
       'Active Students in 20117']
aname=['school_name', 'ad', 'pri_pub', 'active_stu']

name_dict=dict(zip(oname, aname))

In [7]:
df = df.rename( columns=name_dict)

In [8]:
df.school_name.values[0:10]

array(['Archdiocese of Los Angeles', 'EF International Academy',
       'Roman Catholic Diocese of Rockville Centre',
       'Archdiocese of Philadelphia', 'Fairmont Private Schools',
       'CATS Academy Boston', 'Montverde Academy', 'IMG ACADEMY',
       'Archdiocese of Chicago', 'North Broward Preparatory School'],
      dtype=object)

In [9]:
ad = []
for school in df.school_name.values:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': school, 'key': api_key}
    try:
        r = requests.get(url,params=keys)
        result_dic = r.json()
        ad.append(result_dic['results'][0]['geometry']['location'])
    except:
        ad.append(None)

In [10]:
df['latlong']=ad

In [11]:
df.head()

,school_name,ad,pri_pub,active_stu,latlong
0,Archdiocese of Los Angeles,F‐1,"Private, non‐profit","1,625","{'lat': 34.0613259, 'lng': -118.2983873}"
1,EF International Academy,F‐1,"Private, for profit","1,043","{'lat': 41.1068651, 'lng': -73.76860099999999}"
2,Roman Catholic Diocese of Rockville Centre,F‐1,"Private, non‐profit",911,"{'lat': 40.6578232, 'lng': -73.6453058}"
3,Archdiocese of Philadelphia,F‐1,"Private, non‐profit",765,"{'lat': 39.9575083, 'lng': -75.1677624}"
4,Fairmont Private Schools,F‐1,"Private, for profit",716,"{'lat': 33.830368, 'lng': -117.9360249}"


In [12]:
df['lat']=df.latlong.apply(lambda x: x['lat'])

In [13]:
df['lng']=df.latlong.apply(lambda x: x['lng'])

In [14]:
df.head()

,school_name,ad,pri_pub,active_stu,latlong,lat,lng
0,Archdiocese of Los Angeles,F‐1,"Private, non‐profit","1,625","{'lat': 34.0613259, 'lng': -118.2983873}",34.061326,-118.298387
1,EF International Academy,F‐1,"Private, for profit","1,043","{'lat': 41.1068651, 'lng': -73.76860099999999}",41.106865,-73.768601
2,Roman Catholic Diocese of Rockville Centre,F‐1,"Private, non‐profit",911,"{'lat': 40.6578232, 'lng': -73.6453058}",40.657823,-73.645306
3,Archdiocese of Philadelphia,F‐1,"Private, non‐profit",765,"{'lat': 39.9575083, 'lng': -75.1677624}",39.957508,-75.167762
4,Fairmont Private Schools,F‐1,"Private, for profit",716,"{'lat': 33.830368, 'lng': -117.9360249}",33.830368,-117.936025


In [15]:
df = df.drop('latlong', axis = 1)

In [16]:
df.active_stu=df.active_stu.str.replace(',','').astype(int)

In [17]:
df.to_csv('map_.csv', index = False)

In [18]:
df.dtypes

school_name     object
ad              object
pri_pub         object
active_stu       int64
lat            float64
lng            float64
dtype: object

In [19]:
df['private'] = df.pri_pub.str.contains('Private')

In [20]:
df.to_csv('map_2.csv', index = False)

In [21]:
df.head()

,school_name,ad,pri_pub,active_stu,lat,lng,private
0,Archdiocese of Los Angeles,F‐1,"Private, non‐profit",1625,34.061326,-118.298387,True
1,EF International Academy,F‐1,"Private, for profit",1043,41.106865,-73.768601,True
2,Roman Catholic Diocese of Rockville Centre,F‐1,"Private, non‐profit",911,40.657823,-73.645306,True
3,Archdiocese of Philadelphia,F‐1,"Private, non‐profit",765,39.957508,-75.167762,True
4,Fairmont Private Schools,F‐1,"Private, for profit",716,33.830368,-117.936025,True


In [35]:
df['cath'] = df.school_name.str.contains('Archdiocese', 'Catholic')

In [36]:
df['chiris'] = df.school_name.str.contains('Christian')

In [38]:
df['pope'] = df.school_name.str.contains('Pope')

In [39]:
df.head()

,school_name,ad,pri_pub,active_stu,lat,lng,private,cath,chiris,pope
0,Archdiocese of Los Angeles,F‐1,"Private, non‐profit",1625,34.061326,-118.298387,True,True,False,False
1,EF International Academy,F‐1,"Private, for profit",1043,41.106865,-73.768601,True,False,False,False
2,Roman Catholic Diocese of Rockville Centre,F‐1,"Private, non‐profit",911,40.657823,-73.645306,True,False,False,False
3,Archdiocese of Philadelphia,F‐1,"Private, non‐profit",765,39.957508,-75.167762,True,True,False,False
4,Fairmont Private Schools,F‐1,"Private, for profit",716,33.830368,-117.936025,True,False,False,False


In [40]:
df.to_csv('reli.csv', index = False)